In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf

In [ ]:
scaler_features = MinMaxScaler()
X_train = scaler_features.fit_transform(X_train)
X_val = scaler_features.transform(X_val)
X_test = scaler_features.transform(X_test)

scaler_targets = MinMaxScaler()
y_train = scaler_targets.fit_transform(y_train.reshape(-1, 1))
y_val = scaler_targets.transform(y_val.reshape(-1, 1))
y_test = scaler_targets.transform(y_test.reshape(-1, 1))

In [ ]:
tf.random.set_seed(0)

model = Sequential([
    Dense(128, input_dim=10, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1),
])

model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0001))

model.summary()

In [ ]:
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

# define TensorBoard callback
tensorboard_cb = TensorBoard(run_logdir)

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    epochs=300, 
                    batch_size=64, 
                    validation_data=(X_val, y_val), 
                    callbacks=[tensorboard_cb])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006

In [ ]:
df = pd.DataFrame(history.history)
df.plot(figsize=(10,5))
plt.grid(True)
plt.show()

In [ ]:
y_hat = model.predict(X_test)

In [ ]:
y_hat = scaler_targets.inverse_transform(y_hat)
y_test = scaler_targets.inverse_transform(y_test)

In [ ]:
print('RMSE =', np.sqrt(np.mean((y_hat - y_test)**2)))
print('MAE =', np.mean(np.abs(y_hat - y_test)))

In [ ]:
# Visualize predictions vs. true labels
fig = plt.figure(figsize=(8,8))
plt.scatter(y_hat, y_test, color="blue", alpha=0.1)
plt.plot([min(y_hat), max(y_hat)], [min(y_hat), max(y_hat)], color='red', linestyle='--')
plt.ylim([0, 30])
plt.xlim([0, 30])
plt.xlabel("Predictions")
plt.ylabel("True Labels")
plt.title("Evaluation of MLP")
plt.show()